# Vega Lite Examples in Haskell - Multi-View Displays

The overview notebook - `VegaLiteGallery` - describes how 
[`hvega`](http://hackage.haskell.org/package/hvega)
is used to create Vega-Lite visualizations.

-----

## Table of Contents

This notebook represents the [Multi-View DIsplays](https://vega.github.io/vega-lite/examples/#multi-vire-displays)
section of the [Vega-Lite example gallery](https://vega.github.io/vega-lite/examples/).

### [1. Faceting: Trellis Plot or Small Multiples](#Faceting:-Trellis-Plot-or-Small-Multiples)

 - [Trellis Bar Chart](#Trellis-Bar-Chart)
 - [Trellis Stacked Bar Chart](#Trellis-Stacked-Bar-Chart)
 - [Trellis Scatter Plot](#Trellis-Scatter-Plot)
 - [Trellis Histograms](#Trellis-Histograms)
 - [Trellis Scatter Plot Showing Anscombe's Quartet](#Trellis-Scatter-Plot-Showing-Anscombe's-Quartet)
 - [Becker's Barley Trellis Plot](#Becker's-Barley-Trellis-Plot)
 - [Trellis Area](#Trellis-Area)
 - [Trellis Area Plot Showing Annual Temperatures in Seattle](#Trellis-Area-Plot-Showing-Annual-Temperatures-in-Seattle)
 - [Faceted Density Plot](#Faceted-Density-Plot)
 - [Compact Trellis Grid of Bar Charts](#Compact-Trellis-Grid-of-Bar-Charts)
 
### [2. Repeat and Concatenation](#Repeat-and-Concatenation)

 - [Repeat and Layer](#Repeat-and-Layer)
 - [Repeat and Layer to Show Different Weather Measures](#Repeat-and-Layer-to-Show-Different-Weather-Measures)
 - [Vertical Concatenation](#Vertical-Concatenation)
 - [Horizontally Repeated Charts](#Horizontally-Repeated-Charts)
 - Interactive Scatterplot Matrix (repeat of version in the Interactive notebook)
 - [Marginal Histograms](#Marginal-Histograms)
 - [Discretizing Scales](#Discretizing-Scales)
 - [Nested View Concatenation Aligned with Axis minExtent](#Nested-View-Concatenation-Aligned-with-Axis-minExtent)
 - [Population Pyramid](#Population-Pyramid)

---

## Versions

The notebook was last run with the following versions of [`hvega`](https://hackage.haskell.org/package/hvega) and
related modules:

In [1]:
:!ghc-pkg latest ghc
:!ghc-pkg latest ihaskell
:!ghc-pkg latest hvega
:!ghc-pkg latest ihaskell-hvega

ghc-8.8.3

ihaskell-0.10.1.1

hvega-0.9.0.0

ihaskell-hvega-0.3.0.0

As to when it was last run, how about:

In [2]:
import Data.Time (getCurrentTime)
getCurrentTime

2020-05-29 15:29:08.44972479 UTC

## Set up

See the overview notebook for an explanation of this section (it provides code I use to compare the `hvega` output
to the specification given in the Vega-Lite gallery).

In [3]:
:ext OverloadedStrings
:ext QuasiQuotes

In [4]:
-- VegaLite uses these names
import Prelude hiding (filter, lookup, repeat)

import Graphics.Vega.VegaLite

-- IHaskell automatically imports this if the `ihaskell-vega` module is installed
-- import IHaskell.Display.Hvega

-- If you are viewing this in an IHaskell notebook rather than Jupyter Lab,
-- use the following to see the visualizations
--
vlShow = id

In [5]:
import qualified Data.ByteString.Lazy.Char8 as BL8
import qualified Data.HashMap.Strict as HM
import qualified Data.Set as S

import Data.Aeson (Value(Object), encode)
import Data.Aeson.QQ.Simple (aesonQQ)
import Control.Monad (forM_, unless, when)
import Data.Maybe (fromJust)
import System.Directory (removeFile)
import System.Process (readProcess, readProcessWithExitCode)

validate ::
  VLSpec       -- ^ The expected specification
  -> VegaLite  -- ^ The actual visualization
  -> IO ()
validate exp vl = 
  let got = fromVL vl
      put = putStrLn
  in if got == exp
      then put "Okay"
      else do
        let red = "\x1b[31m"
            def = "\x1b[0m"
            
            report m = put (red ++ m ++ def)
            
        report "The visualization and expected specification do not match."
        
        -- assume both objects
        let Object oexp = exp
            Object ogot = got
            kexp = S.fromList (HM.keys oexp)
            kgot = S.fromList (HM.keys ogot)
            kmiss = S.toList (S.difference kexp kgot)
            kextra = S.toList (S.difference kgot kexp)
            keys = S.toList (S.intersection kexp kgot)
            
        unless (null kmiss && null kextra) $ do
          put ""
          report "Keys are different:"
          unless (null kmiss)  $ put ("  Missing: " ++ show kmiss)
          unless (null kextra) $ put ("  Extra  : " ++ show kextra)

        -- this often creates an impressive amount of text for what is
        -- only a small change, which is why it is followed by a call
        -- to debug
        --
        forM_ keys $ \key ->
          let vexp = fromJust (HM.lookup key oexp)
              vgot = fromJust (HM.lookup key ogot)
          in when (vexp /= vgot) $ do
            put ""
            report ("Values are different for " ++ show key)
            put ("  Expected: " ++ show vexp)
            put ("  Found   : " ++ show vgot)
          
        putStrLn ""
        report "The field-level differences are:"
        debug_ exp vl


-- Rather than come up with a way to diff JSON here, rely on `jq` and the trusty
-- `diff` command. This is not written to be robust!
--
debug_ spec vl = do
  let tostr = BL8.unpack . encode
  
  expected <- readProcess "jq" [] (tostr spec)
  got <- readProcess "jq" [] (tostr (fromVL vl))

  let f1 = "expected.json"
      f2 = "got.json"
      
  writeFile f1 expected
  writeFile f2 got
  
  let diffOpts = ["--minimal", f1, f2]
  (_, diff, _) <- readProcessWithExitCode "diff" diffOpts ""
  putStrLn diff
  
  forM_ [f1, f2] removeFile

-----

## Faceting: Trellis Plot or Small Multiples

 - [Trellis Bar Chart](#Trellis-Bar-Chart)
 - [Trellis Stacked Bar Chart](#Trellis-Stacked-Bar-Chart)
 - [Trellis Scatter Plot](#Trellis-Scatter-Plot)
 - [Trellis Histograms](#Trellis-Histograms)
 - [Trellis Scatter Plot Showing Anscombe's Quartet](#Trellis-Scatter-Plot-Showing-Anscombe's-Quartet)
 - [Becker's Barley Trellis Plot](#Becker's-Barley-Trellis-Plot)
 - [Trellis Area](#Trellis-Area)
 - [Faceted Density Plot](#Faceted-Density-Plot)
 
---

### Trellis Bar Chart

From https://vega.github.io/vega-lite/examples/trellis_bar.html

In [6]:
trellisBarSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "A trellis bar chart showing the US population distribution of age groups and gender in 2000.",
  "data": { "url": "data/population.json"},
  "transform": [
    {"filter": "datum.year == 2000"},
    {"calculate": "datum.sex == 2 ? 'Female' : 'Male'", "as": "gender"}
  ],
  "width": {"step": 17},
  "mark": "bar",
  "encoding": {
    "row": {"field": "gender", "type": "nominal"},
    "y": {
      "aggregate": "sum", "field": "people", "type": "quantitative",
      "axis": {"title": "population"}
    },
    "x": {"field": "age", "type": "ordinal"},
    "color": {
      "field": "gender", "type": "nominal",
      "scale": {"range": ["#675193", "#ca8861"]}
    }
  }
}
|]

In [7]:
trellisBar =
    let desc = description "A trellis bar chart showing the US population distribution of age groups and gender in 2000."
        dvals = dataFromUrl "data/population.json" []
        
        trans = transform
                  . filter (FExpr "datum.year == 2000")
                  . calculateAs "datum.sex == 2 ? 'Female' : 'Male'" "gender"
                
        colRange = SRange (RStrings ["#675193", "#ca8861"])
        enc = encoding
                . row [FName "gender", FmType Nominal]
                . position Y [PAggregate Sum, PName "people", PmType Quantitative, PAxis [AxTitle "population"]]
                . position X [PName "age", PmType Ordinal]
                . color [MName "gender", MmType Nominal, MScale [colRange]]
                
    in toVegaLite [desc, dvals, trans [], widthStep 17, mark Bar [], enc []]
    
vlShow trellisBar

In [8]:
validate trellisBarSpec trellisBar

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Trellis Stacked Bar Chart

From https://vega.github.io/vega-lite/examples/trellis_stacked_bar.html

In [9]:
trellisStackedBarSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "data": {"url": "data/barley.json"},
  "mark": "bar",
  "encoding": {
    "column": {"field": "year", "type": "ordinal"},
    "x": {"field": "yield", "type": "quantitative", "aggregate": "sum"},
    "y": {"field": "variety", "type": "nominal"},
    "color": {"field": "site", "type": "nominal"}
  }
}
|]

In [10]:
trellisStackedBar =
    let dvals = dataFromUrl "data/barley.json" []
    
        enc = encoding
                . column [FName "year", FmType Ordinal]
                . position X [PName "yield", PmType Quantitative, PAggregate Sum]
                . position Y [PName "variety", PmType Nominal]
                . color [MName "site", MmType Nominal]
                
    in toVegaLite [dvals, mark Bar [], enc []]
    
vlShow trellisStackedBar

In [11]:
validate trellisStackedBarSpec trellisStackedBar

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Trellis Scatter Plot

From https://vega.github.io/vega-lite/examples/trellis_scatter.html

The JSON can not be re-created with `hvega`, due to how faceted
plots are encoded, as described at
https://github.com/gicentre/elm-vegalite/issues/5. However, the
same plot can be created.

In [12]:
trellisScatterSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "data": {"url": "data/movies.json"},
  "mark": "point",
  "encoding": {
    "facet": {"field": "MPAA_Rating", "type": "ordinal", "columns": 2},
    "x": {"field": "Worldwide_Gross", "type": "quantitative"},
    "y": {"field": "US_DVD_Sales", "type": "quantitative"}
  }
}
|]

In [13]:
trellisScatter =
    let dvals = dataFromUrl "data/movies.json" []
        enc = encoding
                . position X [PName "Worldwide_Gross", PmType Quantitative]
                . position Y [PName "US_DVD_Sales", PmType Quantitative]
        
    in toVegaLite [ dvals
                  , columns 2
                  , facetFlow [FName "MPAA_Rating", FmType Ordinal]
                  , specification (asSpec [mark Point [], enc []])
                  ]
    
vlShow trellisScatter

Although the specification is rather different, as shown below, the visualization it creates is the same as the JSON specification above.

In [14]:
validate trellisScatterSpec trellisScatter

The visualization and expected specification do not match.

Keys are different:
  Missing: ["encoding","mark"]
  Extra  : ["columns","facet","spec"]

The field-level differences are:
2d1
<   "mark": "point",
6,19c5,15
<   "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
<   "encoding": {
<     "facet": {
<       "field": "MPAA_Rating",
<       "columns": 2,
<       "type": "ordinal"
<     },
<     "x": {
<       "field": "Worldwide_Gross",
<       "type": "quantitative"
<     },
<     "y": {
<       "field": "US_DVD_Sales",
<       "type": "quantitative"
---
>   "spec": {
>     "mark": "point",
>     "encoding": {
>       "x": {
>         "field": "Worldwide_Gross",
>         "type": "quantitative"
>       },
>       "y": {
>         "field": "US_DVD_Sales",
>         "type": "quantitative"
>       }
20a17,22
>   },
>   "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
>   "columns": 2,
>   "facet": {
>     "field": "MPAA_Rating",
>     "type": "ordinal"

Return to the [Table of Contents](#Table-of-Contents).

### Trellis Histograms

From https://vega.github.io/vega-lite/examples/trellis_bar_histogram.html

Unlike the previous example, we can directly encode this example in `hvega`.

In [15]:
trellisBarHistogramSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "data": {"url": "data/cars.json"},
  "mark": "bar",
  "encoding": {
    "x": {
      "bin": {"maxbins": 15},
      "field": "Horsepower",
      "type": "quantitative"
    },
    "y": {
      "aggregate": "count",
      "type": "quantitative"
    },
    "row": {"field": "Origin", "type": "nominal"}
  }
}
|]

In [16]:
trellisBarHistogram =
    let dvals = dataFromUrl "data/cars.json" []
        enc = encoding
                . position X [PName "Horsepower", PmType Quantitative, PBin [MaxBins 15]]
                . position Y [PAggregate Count, PmType Quantitative]
                . row [FName "Origin", FmType Nominal]
    
    in toVegaLite [dvals, mark Bar [], enc []]

vlShow trellisBarHistogram

In [17]:
validate trellisBarHistogramSpec trellisBarHistogram

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Trellis Scatter Plot Showing Anscombe's Quartet

From https://vega.github.io/vega-lite/examples/trellis_anscombe.html

In [18]:
trellisAnscombeSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "Anscombe's Quartet",
  "data": {"url": "data/anscombe.json"},
  "mark": "circle",
  "encoding": {
    "column": {"field": "Series", "type": "nominal"},
    "x": {
      "field": "X",
      "type": "quantitative",
      "scale": {"zero": false}
    },
    "y": {
      "field": "Y",
      "type": "quantitative",
      "scale": {"zero": false}
    },
    "opacity": {"value": 1}
  }
}
|]

In [19]:
trellisAnscombe =
    let desc = description "Anscombe's Quartet"
        dvals = dataFromUrl "data/anscombe.json" []
        
        enc = encoding
                . column [FName "Series", FmType Nominal]
                . position X [PName "X", PmType Quantitative, PScale [SZero False]]
                . position Y [PName "Y", PmType Quantitative, PScale [SZero False]]
                . opacity [MNumber 1]
                
    in toVegaLite [desc, dvals, mark Circle [], enc []]

vlShow trellisAnscombe

In [20]:
validate trellisAnscombeSpec trellisAnscombe

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Becker's Barley Trellis Plot

From https://vega.github.io/vega-lite/examples/trellis_barley.html

As with the [Trellis Scatter Plot](#Trellis-Scatter-Plot) example, this visualization has to be created slightly-differently
to the example specification.

In [21]:
trellisBarleySpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "name": "trellis_barley",
  "description": "The Trellis display by Becker et al. helped establish small multiples as a “powerful mechanism for understanding interactions in studies of how a response depends on explanatory variables”. Here we reproduce a trellis of Barley yields from the 1930s, complete with main-effects ordering to facilitate comparison.",
  "data": {"url": "data/barley.json"},
  "mark": "point",
  "height": {"step": 12},
  "encoding": {
    "facet": {
      "field": "site",
      "type": "ordinal",
      "columns": 2,
      "sort": {"op": "median", "field": "yield"}
    },
    "x": {
      "aggregate": "median",
      "field": "yield",
      "type": "quantitative",
      "scale": {"zero": false}
    },
    "y": {
      "field": "variety",
      "type": "ordinal",
      "sort": "-x"
    },
    "color": {"field": "year", "type": "nominal"}
  }
}
|]

In [22]:
trellisBarley =
    let desc = "The Trellis display by Becker et al. helped establish small multiples as a " <>
               "“powerful mechanism for understanding interactions in studies of how a response " <>
               "depends on explanatory variables”. Here we reproduce a trellis of Barley yields " <>
               "from the 1930s, complete with main-effects ordering to facilitate comparison."
        dvals = dataFromUrl "data/barley.json"
        markOpts = mark Point []
        
        cols = columns 2
        fct = facetFlow [FName "site", FmType Ordinal, FSort [ByFieldOp "yield" Median]]
        
        enc = encoding
                 . position X [PName "yield", PAggregate Median, PmType Quantitative, PScale [SZero False]]
                 . position Y [PName "variety", PmType Ordinal, PSort [Descending]]
                 . color [MName "year", MmType Nominal]
        
    in toVegaLite [ name "trellis_barley"
                  , description desc
                  , dvals []
                  , cols
                  , fct
                  , specification (asSpec [markOpts, heightStep 12, enc []])
                  ]
    
    
vlShow trellisBarley

In [23]:
validate trellisBarleySpec trellisBarley

The visualization and expected specification do not match.

Keys are different:
  Missing: ["encoding","height","mark"]
  Extra  : ["columns","facet","spec"]

The field-level differences are:
2,5d1
<   "height": {
<     "step": 12
<   },
<   "mark": "point",
10,14c6,8
<   "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
<   "encoding": {
<     "color": {
<       "field": "year",
<       "type": "nominal"
---
>   "spec": {
>     "height": {
>       "step": 12
16,20c10,14
<     "facet": {
<       "field": "site",
<       "sort": {
<         "op": "median",
<         "field": "yield"
---
>     "mark": "point",
>     "encoding": {
>       "color": {
>         "field": "year",
>         "type": "nominal"
22,28c16,22
<       "columns": 2,
<       "type": "ordinal"
<     },
<     "x": {
<       "field": "yield",
<       "scale": {
<         "zero": false
---
>       "x": {
>         "field": "yield",
>         "scale": {
>           "zero": false
>         },
>         "aggregate

Return to the [Table of Contents](#Table-of-Contents).

### Trellis Area

From https://vega.github.io/vega-lite/examples/trellis_area.html

In [24]:
trellisAreaSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "Stock prices of four large companies as a small multiples of area charts.",
  "transform": [
      {
          "filter": "datum.symbol !== 'GOOG'"
      }
  ],
  "width": 300,
  "height": 40,
  "data": {
      "url": "data/stocks.csv"
  },
  "mark": "area",
  "encoding": {
      "x": {
          "field": "date",
          "type": "temporal",
          "axis": {
              "title": "Time",
              "grid": false
          }
      },
      "y": {
          "field": "price",
          "type": "quantitative",
          "axis": {
              "title": "Price",
              "grid": false
          }
      },
      "color": {
          "field": "symbol",
          "type": "nominal",
          "legend": null
      },
      "row": {
          "field": "symbol",
          "type": "nominal",
          "header": {"title": "Symbol"}
      }
  }
}
|]

In [25]:
trellisArea =
    let desc = "Stock prices of four large companies as a small multiples of area charts."
    
        xEnc = position X [PName "date", PmType Temporal, PAxis [AxTitle "Time", AxGrid False]]
        yEnc = position Y [PName "price", PmType Quantitative, PAxis [AxTitle "Price", AxGrid False]]
        cEnc = color [MName "symbol", MmType Nominal, MLegend []]
        rEnc = row [FName "symbol", FmType Nominal, FHeader [HTitle "Symbol"]]
        
    in toVegaLite [ description desc
                  , transform (filter (FExpr "datum.symbol !== 'GOOG'") [])
                  , width 300
                  , height 40
                  , dataFromUrl "data/stocks.csv" []
                  , mark Area []
                  , encoding . xEnc . yEnc . cEnc . rEnc $ []
                  ]
    
vlShow trellisArea

In [26]:
validate trellisAreaSpec trellisArea

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Trellis Area Plot Showing Annual Temperatures in Seattle

From https://vega.github.io/vega-lite/examples/trellis_area_seattle.html

With `hvega 0.7.0.0` I am unable to re-create the specification exactly (as discussed below) but the visualization does look the same.

In [27]:
trellisAreaSeattleSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "Average temperatures in Seattle, broken down by hour. Derived from [Seattle Annual Temperate](https://vega.github.io/vega/examples/annual-temperature/) example from the Vega example gallery.",
  "title": "Seattle Annual Temperatures",
  "data": {"url": "data/seattle-temps.csv"},
  "transform": [
    {"calculate": "(hours(datum.date) + 18) % 24", "as": "order"}
  ],
  "spacing": {"row": 1},
  "facet": {
    "row": {
      "field": "date",
      "timeUnit": "hours",
      "type": "nominal",
      "sort": {"field": "order"},
      "header": {
        "labelAngle": 0,
        "labelPadding": 2,
        "titlePadding": -4,
        "labelAlign": "left",
        "labelExpr": "hours(datum.value) == 0 ? 'Midnight' : hours(datum.value) == 12 ? 'Noon' : timeFormat(datum.value, '%I:%M %p')"
      }
    }
  },
  "spec": {
    "width": 800,
    "height": 25,
    "view": {"stroke": null},
    "mark": "area",
    "encoding": {
      "x": {
        "field": "date",
        "type": "temporal",
        "title": "Month",
        "axis": {"format": "%b"}
      },
      "y": {
        "field": "temp",
        "type": "quantitative",
        "scale": {"zero": false},
        "axis": {"title": null, "labels": false, "ticks": false}
      }
    }
  },
  "config": {"axis": {"grid": false, "domain": false}}
}
|]

In [28]:
trellisAreaSeattle =
  let desc = "Average temperatures in Seattle, broken down by hour. Derived from [Seattle Annual Temperate](https://vega.github.io/vega/examples/annual-temperature/) example from the Vega example gallery."

      ylabels = "hours(datum.value) == 0 ? 'Midnight' : hours(datum.value) == 12 ? 'Noon' : timeFormat(datum.value, '%I:%M %p')"

      plot = asSpec [ width 800
                    , height 25
                    , viewBackground [VBNoStroke]
                    , mark Area []
                    , encoding
                      . position X [ PName "date"
                                   , PmType Temporal
                                   , PTitle "Month"
                                   , PAxis [AxFormat "%b"]
                                   ]
                      . position Y [ PName "temp"
                                   , PmType Quantitative
                                   , PScale [SZero False]
                                   , PAxis [AxNoTitle, AxLabels False, AxTicks False]
                                   ]
                      $ []
                    ]

      vs = [ description desc
           , title "Seattle Annual Temperatures" []
           , dataFromUrl "data/seattle-temps.csv" []
           , transform
             (calculateAs "(hours(datum.date) + 18) % 24" "order" [])
           , spacing 1
           , configure (configuration (Axis [Grid False, Domain False]) [])
           , facet [ RowBy [ FName "date"
                           , FmType Nominal
                           , FTimeUnit Hours
                           , FSort [ByFieldOp "order" Max]
                           , FHeader [ HLabelAngle 0
                                     , HLabelPadding 2
                                     , HTitlePadding (-4)
                                     , HLabelAlign AlignLeft
                                     , HLabelExpr ylabels
                                     ]
                           ]
                   ]
           , specification plot
           ]

  in toVegaLite vs
  
vlShow trellisAreaSeattle

There are two known differences between `hvega` and the JSON specification from the Vega-Lite examples page:

 1. the `spacing` can not be restricted to just the `row` (in this
    case there are no columns so this is not a problem)
    
 2. the facet ordering (on the `"order"` field) requires an `Operation`
    in `hvega` which is not needed (so I chose `Max` as there are no
    repeated values so it should not change the result)

In [29]:
validate trellisAreaSeattleSpec trellisAreaSeattle

The visualization and expected specification do not match.

Values are different for "facet"
  Expected: Object (fromList [("row",Object (fromList [("field",String "date"),("timeUnit",String "hours"),("header",Object (fromList [("labelAngle",Number 0.0),("labelExpr",String "hours(datum.value) == 0 ? 'Midnight' : hours(datum.value) == 12 ? 'Noon' : timeFormat(datum.value, '%I:%M %p')"),("titlePadding",Number -4.0),("labelPadding",Number 2.0),("labelAlign",String "left")])),("sort",Object (fromList [("field",String "order")])),("type",String "nominal")]))])
  Found   : Object (fromList [("row",Object (fromList [("field",String "date"),("timeUnit",String "hours"),("header",Object (fromList [("labelAngle",Number 0.0),("labelExpr",String "hours(datum.value) == 0 ? 'Midnight' : hours(datum.value) == 12 ? 'Noon' : timeFormat(datum.value, '%I:%M %p')"),("titlePadding",Number -4.0),("labelPadding",Number 2.0),("labelAlign",String "left")])),("sort",Object (fromList [("op",String "max"),("field"

Return to the [Table of Contents](#Table-of-Contents).

### Faceted Density Plot

From https://vega.github.io/vega-lite/examples/area_density_facet.html

In [30]:
areaDensityFacetSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "width": 300,
  "height": 50,
  "data": {"url": "data/iris.json"},
  "mark": "area",
  "transform": [
    {
      "fold": [
        "petalWidth",
        "petalLength",
        "sepalWidth",
        "sepalLength"
      ],
      "as": ["organ", "value"]
    },
    {
      "density": "value",
      "bandwidth": 0.3,
      "groupby": ["organ"]
    }
  ],
  "encoding": {
    "x": {
      "field": "value",
      "type": "quantitative"
    },
    "y": {
      "field": "density",
      "type": "quantitative"
    },
    "row": {
      "field": "organ",
      "type": "nominal"
    }
  }
}
|]

In [31]:
areaDensityFacet =
    let dnOpts = [DnBandwidth 0.3, DnGroupBy ["organ"]]
        fields = ["petalWidth", "petalLength", "sepalWidth", "sepalLength"]
        
        v = [ width 300
            , height 50
            , dataFromUrl "data/iris.json" []
            , mark Area []
            , transform
              . foldAs fields "organ" "value"
              . density "value" dnOpts
              $ []
            , encoding
              . position X [PName "value", PmType Quantitative]
              . position Y [PName "density", PmType Quantitative]
              . row [FName "organ", FmType Nominal]
              $ []
            ]
            
    in toVegaLite v
    
vlShow areaDensityFacet

In [32]:
validate areaDensityFacetSpec areaDensityFacet

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Compact Trellis Grid of Bar Charts

From https://vega.github.io/vega-lite/examples/facet_grid_bar.html

In [33]:
facetGridBarSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "A simple grid of bar charts to compare performance data.",
  "data": {
    "values": [
      {"a": "a1", "b": "b1", "c": "x", "p": "0.14"},
      {"a": "a1", "b": "b1", "c": "y", "p": "0.60"},
      {"a": "a1", "b": "b1", "c": "z", "p": "0.03"},
      {"a": "a1", "b": "b2", "c": "x", "p": "0.80"},
      {"a": "a1", "b": "b2", "c": "y", "p": "0.38"},
      {"a": "a1", "b": "b2", "c": "z", "p": "0.55"},
      {"a": "a1", "b": "b3", "c": "x", "p": "0.11"},
      {"a": "a1", "b": "b3", "c": "y", "p": "0.58"},
      {"a": "a1", "b": "b3", "c": "z", "p": "0.79"},
      {"a": "a2", "b": "b1", "c": "x", "p": "0.83"},
      {"a": "a2", "b": "b1", "c": "y", "p": "0.87"},
      {"a": "a2", "b": "b1", "c": "z", "p": "0.67"},
      {"a": "a2", "b": "b2", "c": "x", "p": "0.97"},
      {"a": "a2", "b": "b2", "c": "y", "p": "0.84"},
      {"a": "a2", "b": "b2", "c": "z", "p": "0.90"},
      {"a": "a2", "b": "b3", "c": "x", "p": "0.74"},
      {"a": "a2", "b": "b3", "c": "y", "p": "0.64"},
      {"a": "a2", "b": "b3", "c": "z", "p": "0.19"},
      {"a": "a3", "b": "b1", "c": "x", "p": "0.57"},
      {"a": "a3", "b": "b1", "c": "y", "p": "0.35"},
      {"a": "a3", "b": "b1", "c": "z", "p": "0.49"},
      {"a": "a3", "b": "b2", "c": "x", "p": "0.91"},
      {"a": "a3", "b": "b2", "c": "y", "p": "0.38"},
      {"a": "a3", "b": "b2", "c": "z", "p": "0.91"},
      {"a": "a3", "b": "b3", "c": "x", "p": "0.99"},
      {"a": "a3", "b": "b3", "c": "y", "p": "0.80"},
      {"a": "a3", "b": "b3", "c": "z", "p": "0.37"}
    ]
  },
  "width": 60,
  "height": {"step": 8},
  "spacing": 5,
  "mark": "bar",
  "encoding": {
    "y": {"field": "c", "type": "nominal", "axis": null},
    "x": {
      "field": "p",
      "type": "quantitative",
      "axis": {"format": "%"},
      "title": null
    },
    "color": {
      "field": "c",
      "type": "nominal",
      "legend": {"orient": "bottom", "titleOrient": "left"},
      "title": "settings"
    },
    "row": {"field": "a", "type": "nominal",
      "title": "Factor A", "header": {"labelAngle": 0}},
    "column": {"field": "b", "type": "nominal",
      "title": "Factor B"}
  }
}
|]

In [34]:
facetGridBar =
    let desc = "A simple grid of bar charts to compare performance data."

        dvals = dataFromColumns []
                . dataColumn "a" (Strings a)
                . dataColumn "b" (Strings b)
                . dataColumn "c" (Strings c)
                . dataColumn "p" (Strings p)
                
        a = [ "a1", "a1", "a1", "a1", "a1", "a1", "a1", "a1", "a1"
            , "a2", "a2", "a2", "a2", "a2", "a2", "a2", "a2", "a2"
            , "a3", "a3", "a3", "a3", "a3", "a3", "a3", "a3", "a3"
            ]
        b = [ "b1", "b1", "b1", "b2", "b2", "b2", "b3", "b3", "b3"
            , "b1", "b1", "b1", "b2", "b2", "b2", "b3", "b3", "b3"
            , "b1", "b1", "b1", "b2", "b2", "b2", "b3", "b3", "b3"
            ]
        c = [ "x", "y", "z", "x", "y", "z", "x", "y", "z"
            , "x", "y", "z", "x", "y", "z", "x", "y", "z"
            , "x", "y", "z", "x", "y", "z", "x", "y", "z"
            ]
        p = [ "0.14", "0.60", "0.03", "0.80", "0.38", "0.55", "0.11", "0.58", "0.79"
            , "0.83", "0.87", "0.67", "0.97", "0.84", "0.90", "0.74", "0.64", "0.19"
            , "0.57", "0.35", "0.49", "0.91", "0.38", "0.91", "0.99", "0.80", "0.37"
            ]

        enc = encoding
              . position Y [PName "c", PmType Nominal, PAxis []]
              . position X [PName "p", PmType Quantitative, PAxis [AxFormat "%"], PNoTitle]
              . color [ MName "c"
                      , MmType Nominal
                      , MLegend [LOrient LOBottom, LTitleOrient SLeft]
                      , MTitle "settings"
                      ]
              . row [ FName "a"
                    , FmType Nominal
                    , FTitle "Factor A"
                    , FHeader [HLabelAngle 0]
                    ]
              . column [ FName "b"
                       , FmType Nominal
                       , FTitle "Factor B"
                       ]
        
    in toVegaLite [ description desc
                  , dvals []
                  , width 60
                  , heightStep 8
                  , spacing 5
                  , mark Bar []
                  , enc []
                  ]

vlShow facetGridBar

In [35]:
validate facetGridBarSpec facetGridBar

Okay

Return to the [Table of Contents](#Table-of-Contents).

-----
 
## Repeat and Concatenation

 - [Repeat and Layer](#Repeat-and-Layer)
 - [Repeat and Layer to Show Different Weather Measures](#Repeat-and-Layer-to-Show-Different-Weather-Measures)
 - [Vertical Concatenation](#Vertical-Concatenation)
 - [Horizontally Repeated Charts](#Horizontally-Repeated-Charts)
 - Interactive Scatterplot Matrix (repeat of version in the Interactive notebook)
 - [Marginal Histograms](#Marginal-Histograms)
 - [Discretizing Scales](#Discretizing-Scales)
 - [Nested View Concatenation Aligned with Axis minExtent](#Nested-View-Concatenation-Aligned-with-Axis-minExtent)
 - [Population Pyramid](#Population-Pyramid)

---

### Repeat and Layer

https://vega.github.io/vega-lite/examples/repeat_layer.html

In [36]:
repeatLayerSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "data": {
    "url": "data/movies.json"
  },
  "repeat": {
    "layer": ["US_Gross", "Worldwide_Gross"]
  },
  "spec": {
    "mark": "line",
    "encoding": {
      "x": {
        "bin": true,
        "field": "IMDB_Rating",
        "type": "quantitative"
      },
      "y": {
        "aggregate": "mean",
        "field": {"repeat": "layer"},
        "type": "quantitative",
        "title": "Mean of US and Worldwide Gross"
      },
      "color": {
        "datum": {"repeat": "layer"},
        "type": "nominal"
      }
    }
  }
}
|]

In [37]:
repeatLayer =
    let dvals = dataFromUrl "data/movies.json" []

        plot = [ mark Line []
               , encoding
                 . position X [ PName "IMDB_Rating"
                              , PmType Quantitative
                              , PBin []
                              ]
                 . position Y [ PRepeat Layer
                              , PmType Quantitative
                              , PAggregate Mean
                              , PTitle "Mean of US and Worldwide Gross"
                              ]
                 . color [MRepeatDatum Layer, MmType Nominal]
                 $ []
               ]
               
    in toVegaLite [ dvals
                  , repeat [LayerFields ["US_Gross", "Worldwide_Gross"]]
                  , specification (asSpec plot)
                  ]

vlShow repeatLayer

In [38]:
validate repeatLayerSpec repeatLayer

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Repeat and Layer to Show Different Weather Measures

From https://vega.github.io/vega-lite/examples/repeat_layer.html but this has now
changed, but I quite like this examle so I'm leaving it as is.

In [39]:
oldRepeatLayerSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "Summarized and per year weather information for Seatle and New York.",
  "data": {"url": "data/weather.csv"},
  "repeat": {"column": ["temp_max", "precipitation", "wind"]},
  "spec": {
    "layer": [
      {
        "mark": "line",
        "encoding": {
          "y": {
            "aggregate": "mean",
            "field": {"repeat": "column"},
            "type": "quantitative"
          },
          "x": {
            "timeUnit": "month",
            "field": "date",
            "type": "ordinal"
          },
          "detail": {
            "timeUnit": "year",
            "type": "temporal",
            "field": "date"
          },
          "color": {"type": "nominal", "field": "location"},
          "opacity": {"value": 0.2}
        }
      },
      {
        "mark": "line",
        "encoding": {
          "y": {
            "aggregate": "mean",
            "field": {"repeat": "column"},
            "type": "quantitative"
          },
          "x": {
            "timeUnit": "month",
            "field": "date",
            "type": "ordinal"
          },
          "color": {"type": "nominal", "field": "location"}
        }
      }
    ]
  }
}
|]

In [40]:
oldRepeatLayer =
    let desc = "Summarized and per year weather information for Seatle and New York."
    
        lineOpts = mark Line []
        
        baseEnc = encoding
                     . position X [PName "date", PmType Ordinal, PTimeUnit Month]
                     . position Y [PAggregate Mean, PmType Quantitative, PRepeat Column]
                     . color [MName "location", MmType Nominal]
                     
        extraEnc = detail [DName "date", DmType Temporal, DTimeUnit Year]
                   . opacity [MNumber 0.2]
        
        byYear = [lineOpts, (baseEnc . extraEnc) []]
        combined = [lineOpts, baseEnc []]
        
        specs = layer (map asSpec [byYear, combined])
        
    in toVegaLite [ description desc, dataFromUrl "data/weather.csv" []
                  , repeat [ColumnFields ["temp_max", "precipitation", "wind"]]
                  , specification (asSpec [specs])
                  ]
    
vlShow oldRepeatLayer

In [41]:
validate oldRepeatLayerSpec oldRepeatLayer

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Vertical Concatenation

From https://vega.github.io/vega-lite/examples/vconcat_weather.html

In [42]:
vconcatWeatherSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "Two vertically concatenated charts that show a histogram of precipitation in Seattle and the relationship between min and max temperature.",
  "data": {
    "url": "data/weather.csv"
  },
  "transform": [{
    "filter": "datum.location === 'Seattle'"
  }],
  "vconcat": [
    {
      "mark": "bar",
      "encoding": {
        "x": {
          "timeUnit": "month",
          "field": "date",
          "type": "ordinal"
        },
        "y": {
          "aggregate": "mean",
          "field": "precipitation",
          "type": "quantitative"
        }
      }
    },
    {
      "mark": "point",
      "encoding": {
        "x": {
          "field": "temp_min",
          "type": "quantitative",
          "bin": true
        },
        "y": {
          "field": "temp_max",
          "type": "quantitative",
          "bin": true
        },
        "size": {
          "aggregate": "count",
          "type": "quantitative"
        }
      }
    }
  ]
}
|]

In [43]:
vconcatWeather =
    let desc = "Two vertically concatenated charts that show a histogram of precipitation in Seattle and the relationship between min and max temperature."
    
        encRain = encoding
                    . position X [PName "date", PmType Ordinal, PTimeUnit Month]
                    . position Y [PName "precipitation", PmType Quantitative, PAggregate Mean]
                    
        encTemp = encoding
                    . position X [PName "temp_min", PmType Quantitative, PBin []]
                    . position Y [PName "temp_max", PmType Quantitative, PBin []]
                    . size [MAggregate Count, MmType Quantitative]
                    
        rain = [mark Bar [], encRain []]
        temp = [mark Point [], encTemp []]
        plots = map asSpec [rain, temp]
        
    in toVegaLite [ description desc
                  , dataFromUrl "data/weather.csv" []
                  , transform . filter (FExpr "datum.location === 'Seattle'") $ []
                  , vConcat plots
                  ]

vlShow vconcatWeather

In [44]:
validate vconcatWeatherSpec vconcatWeather

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Horizontally Repeated Charts

From https://vega.github.io/vega-lite/examples/repeat_histogram.html

In [45]:
repeatHistogramSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "repeat": ["Horsepower", "Miles_per_Gallon", "Acceleration", "Displacement"],
  "columns": 2,
  "spec": {
    "data": {"url": "data/cars.json"},
    "mark": "bar",
    "encoding": {
      "x": {
        "field": {"repeat": "repeat"},
        "bin": true,
        "type": "quantitative"
      },
      "y": {"aggregate": "count", "type": "quantitative"},
      "color": {"field": "Origin", "type": "nominal"}
    }
  }
}
|]

In [46]:
repeatHistogram =
    let cols = ["Horsepower", "Miles_per_Gallon", "Acceleration", "Displacement"]
    
        enc = encoding
                . position X [PRepeat Flow, PBin [], PmType Quantitative]
                . position Y [PAggregate Count, PmType Quantitative]
                . color [MName "Origin", MmType Nominal]
                
        plot = [ dataFromUrl "data/cars.json" []
               , mark Bar []
               , enc []
               ]
    in toVegaLite [ repeatFlow cols
                  , columns 2
                  , specification (asSpec plot)
                  ]
                  
vlShow repeatHistogram

In [47]:
validate repeatHistogramSpec repeatHistogram

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Marginal Histograms

From https://vega.github.io/vega-lite/examples/concat_marginal_histograms.html

In [48]:
concatMarginalHistogramsSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "data": {"url": "data/movies.json"},
  "spacing": 15,
  "bounds": "flush",
  "vconcat": [{
    "mark": "bar",
    "height": 60,
    "encoding": {
      "x": {
        "bin": true,
        "field": "IMDB_Rating",
        "type": "quantitative",
        "axis": null
      },
      "y": {
        "aggregate": "count",
        "type": "quantitative",
        "scale": {
          "domain": [0,1000]
        },
        "title": ""
      }
    }
  }, {
    "spacing": 15,
    "bounds": "flush",
    "hconcat": [{
      "mark": "rect",
      "encoding": {
        "x": {
          "bin": true,
          "field": "IMDB_Rating",
          "type": "quantitative"
        },
        "y": {
          "bin": true,
          "field": "Rotten_Tomatoes_Rating",
          "type": "quantitative"
        },
        "color": {
          "aggregate": "count",
          "type": "quantitative"
        }
      }
    }, {
      "mark": "bar",
      "width": 60,
      "encoding": {
        "y": {
          "bin": true,
          "field": "Rotten_Tomatoes_Rating",
          "type": "quantitative",
          "axis": null
        },
        "x": {
          "aggregate": "count",
          "type": "quantitative",
          "scale": {
            "domain": [0,1000]
          },
          "title": ""
        }
      }
    }]
  }],
  "config": {
    "view": {
      "stroke": "transparent"
    }
  }
}
|]

In [49]:
concatMarginalHistograms =
    let plotBounds = bounds Flush
        plotSpacing = spacing 15
    
        binAxis dir name extra = position dir ([PName name, PmType Quantitative, PBin []] <> extra)
        countAxis dir = position dir [PAggregate Count, PmType Quantitative, PTitle "", PScale [SDomain (DNumbers [0, 1000])]]


        histIMDB = [ mark Bar []
                   , height 60
                   , encoding
                       . binAxis X "IMDB_Rating" [PAxis []]
                       . countAxis Y
                       $ []
                       
                   ]
        histRT = [ mark Bar []
                 , width 60
                 , encoding
                      . binAxis Y "Rotten_Tomatoes_Rating" [PAxis []]
                      . countAxis X
                      $ []
                 ]

        mainPlot = [ mark Rect []
                   , encoding
                       . binAxis X "IMDB_Rating" []
                       . binAxis Y "Rotten_Tomatoes_Rating" []
                       . color [MAggregate Count, MmType Quantitative]
                       $ []
                   ]
        
        hPlots = map asSpec [mainPlot, histRT]
        
        plots = map asSpec [histIMDB, [plotSpacing, plotBounds, hConcat hPlots]]
    
    in toVegaLite [ dataFromUrl "data/movies.json" []
                  , plotBounds
                  , plotSpacing
                  , vConcat plots
                  , configure (configuration (ViewStyle [ViewStroke "transparent"]) [])
                  ]

vlShow concatMarginalHistograms

In [50]:
validate concatMarginalHistogramsSpec concatMarginalHistograms

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Discretizing Scales

From https://vega.github.io/vega-lite/examples/concat_bar_scales_discretize.html

In [51]:
concatBarScalesDiscretizeSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "Horizontally concatenated charts that show different types of discretizing scales.",
  "data": {
    "values": [
      {"a": "A", "b": 28},
      {"a": "B", "b": 55},
      {"a": "C", "b": 43},
      {"a": "D", "b": 91},
      {"a": "E", "b": 81},
      {"a": "F", "b": 53},
      {"a": "G", "b": 19},
      {"a": "H", "b": 87},
      {"a": "I", "b": 52}
    ]
  },
  "hconcat": [
    {
      "mark": "circle",
      "encoding": {
        "y": {
          "field": "b",
          "type": "nominal",
          "sort": null,
          "axis": {
            "ticks": false,
            "domain": false,
            "title": null
          }
        },
        "size": {
          "field": "b",
          "type": "quantitative",
          "scale": {
            "type": "quantize"
          }
        },
        "color": {
          "field": "b",
          "type": "quantitative",
          "scale": {
            "type": "quantize",
            "zero": true
          },
          "legend": {
            "title": "Quantize"
          }
        }
      }
    },
    {
      "mark": "circle",
      "encoding": {
        "y": {
          "field": "b",
          "type": "nominal",
          "sort": null,
          "axis": {
            "ticks": false,
            "domain": false,
            "title": null
          }
        },
        "size": {
          "field": "b",
          "type": "quantitative",
          "scale": {
            "type": "quantile",
            "range": [80, 160, 240, 320, 400]
          }
        },
        "color": {
          "field": "b",
          "type": "quantitative",
          "scale": {
            "type": "quantile",
            "scheme": "magma"
          },
          "legend": {
            "format": "d",
            "title": "Quantile"
          }
        }
      }
    },
    {
      "mark": "circle",
      "encoding": {
        "y": {
          "field": "b",
          "type": "nominal",
          "sort": null,
          "axis": {
            "ticks": false,
            "domain": false,
            "title": null
          }
        },
        "size": {
          "field": "b",
          "type": "quantitative",
          "scale": {
            "type": "threshold",
            "domain": [30, 70],
            "range": [80, 200, 320]
          }
        },
        "color": {
          "field": "b",
          "type": "quantitative",
          "scale": {
            "type": "threshold",
            "domain": [30, 70],
            "scheme": "viridis"
          },
          "legend": {
            "title": "Threshold"
          }
        }
      }
    }
  ],
  "resolve": {
    "scale": {
      "color": "independent",
      "size": "independent"
    }
  }
}
|]

In [52]:
concatBarScalesDiscretize =
    let dvals = dataFromColumns []
                  . dataColumn "a" (Strings ["A", "B", "C", "D", "E", "F", "G", "H", "I"])
                  . dataColumn "b" (Numbers [28, 55, 43, 91, 81, 53, 19, 87, 52])


        yEnc = position Y [PName "b", PmType Nominal, PSort [], PAxis [AxTicks False, AxDomain False, AxNoTitle]]

        sizeEnc stype sextra = size [MName "b", MmType Quantitative, MScale ([SType stype] <> sextra)]
        
        colorEnc title stype sextra lextra =
            color [ MName "b"
                  , MmType Quantitative
                  , MScale ([SType stype] <> sextra)
                  , MLegend ([LTitle title] <> lextra)
                               ]

        quantize = [ mark Circle []
                   , encoding
                       . yEnc
                       . sizeEnc ScQuantize []
                       . colorEnc "Quantize" ScQuantize [SZero True] []
                       $ []
                   ]
        quantile = [ mark Circle []
                   , encoding
                       . yEnc
                       . sizeEnc ScQuantile [SRange (RNumbers [80, 160, 240, 320, 400])]
                       . colorEnc "Quantile" ScQuantile [SScheme "magma" []] [LFormat "d"]
                       $ []
                   ]
        threshold = [ mark Circle []
                    , encoding
                        . yEnc
                        . sizeEnc ScThreshold [ SDomain (DNumbers [30, 70])
                                              , SRange (RNumbers [80, 200, 320])
                                              ]
                        . colorEnc "Threshold" ScThreshold [SScheme "viridis" [], SDomain (DNumbers [30, 70])] []
                        $ []
                    ]

        plots = map asSpec [quantize, quantile, threshold]
        
    in toVegaLite [ description "Horizontally concatenated charts that show different types of discretizing scales."
                  , dvals []
                  , hConcat plots
                  , resolve (resolution (RScale [(ChColor, Independent), (ChSize, Independent)]) [])
                  ]

vlShow concatBarScalesDiscretize

In [53]:
validate concatBarScalesDiscretizeSpec concatBarScalesDiscretize

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Nested View Concatenation Aligned with Axis minExtent

From https://vega.github.io/vega-lite/examples/nested_concat_align.html

In [54]:
nestedConcatAlignSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "Nested concatenation aligned by setting axis minExtent",
  "data": {
    "url": "data/movies.json"
  },
  "vconcat": [{
    "title": "Ratings",
    "repeat": {"column": ["Rotten_Tomatoes_Rating", "IMDB_Rating"]},
    "spec": {
      "width": 150,
      "height": 50,
      "mark": "bar",
      "encoding": {
        "x": {
          "field": {"repeat": "column"},
          "bin": {"maxbins": 20},
          "type": "quantitative"
        },
        "y": {"aggregate": "count", "type": "quantitative"}
      }
    }
  },{
    "title": "Gross",
    "repeat": {"column": ["US_Gross", "Worldwide_Gross"]},
    "spec": {
      "width": 150,
      "height": 50,
      "encoding": {
        "x": {
          "field": {"repeat": "column"},
          "bin": {"maxbins": 20},
          "type": "quantitative"
        },
        "y": {"aggregate": "count", "type": "quantitative"}
      },
      "mark": "bar"
    }
  }],
  "config": {
    "countTitle": "Count",
    "axisX": {"titleLimit": 150},
    "axisY": {
      "minExtent": 40
    }
  }
}
|]

In [55]:
nestedConcatAlign =
    let enc = encoding
                . position X [PRepeat Column, PBin [MaxBins 20], PmType Quantitative]
                . position Y [PAggregate Count, PmType Quantitative]
    
        rspec = [ width 150
                , height 50
                , mark Bar []
                , enc []
                ]
        ratings = [ title "Ratings" []
                  , repeat [ColumnFields ["Rotten_Tomatoes_Rating", "IMDB_Rating"]]
                  , specification (asSpec rspec)
                  ]
        
        gspec = [ width 150
                , height 50
                , mark Bar []
                , enc []
                ]
        gross = [ title "Gross" []
                , repeat [ColumnFields ["US_Gross", "Worldwide_Gross"]]
                , specification (asSpec gspec)
                ]
        
        plots = map asSpec [ratings, gross]
        
        conf = configure
                 . configuration (CountTitleStyle "Count")
                 . configuration (AxisX [TitleLimit 150])
                 . configuration (AxisY [MinExtent 40])
                
    in toVegaLite [ description "Nested concatenation aligned by setting axis minExtent"
                  , dataFromUrl "data/movies.json" []
                  , vConcat plots
                  , conf []
                  ]

vlShow nestedConcatAlign

In [56]:
validate nestedConcatAlignSpec nestedConcatAlign

Okay

Return to the [Table of Contents](#Table-of-Contents).

### Population Pyramid

From https://vega.github.io/vega-lite/examples/concat_population_pyramid.html

This is very-similar to the "Bar Chart" version in the Single View
notebook (https://vega.github.io/vega-lite/examples/bar_diverging_stack_population_pyramid.html).

In [57]:
concatPopulationPyramidSpec = [aesonQQ|
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "description": "A population pyramid for the US in 2000.",
  "data": { "url": "data/population.json"},
  "transform": [
    {"filter": "datum.year == 2000"},
    {"calculate": "datum.sex == 2 ? 'Female' : 'Male'", "as": "gender"}
  ],
  "spacing": 0,
  "hconcat": [{
    "transform": [{
      "filter": {"field": "gender", "equal": "Female"}
    }],
    "title": "Female",
    "mark": "bar",
    "encoding": {
      "y": {
        "field": "age", "type": "ordinal",
        "axis": null, "sort": "descending"
      },
      "x": {
        "aggregate": "sum", "field": "people", "type": "quantitative",
        "axis": {"title": "population", "format": "s"},
        "sort": "descending"
      },
      "color": {
        "field": "gender", "type": "nominal",
        "scale": {"range": ["#675193", "#ca8861"]},
        "legend": null
      }
    }
  }, {
    "width": 20,
    "view": {"stroke": null},
    "mark": {
      "type": "text",
      "align": "center"
    },
    "encoding": {
      "y": {"field": "age", "type": "ordinal", "axis": null, "sort": "descending"},
      "text": {"field": "age", "type": "quantitative"}
    }
  }, {
    "transform": [{
      "filter": {"field": "gender", "equal": "Male"}
    }],
    "title": "Male",
    "mark": "bar",
    "encoding": {
      "y": {
        "field": "age", "type": "ordinal", "title": null,
        "axis": null, "sort": "descending"
      },
      "x": {
        "aggregate": "sum", "field": "people", "type": "quantitative",
        "axis": {"title": "population", "format": "s"}
      },
      "color": {
        "field": "gender", "type": "nominal",
        "legend": null
      }
    }
  }],
  "config": {
    "view": {"stroke": null},
    "axis": {"grid": false}
  }
}
|]

In [58]:
concatPopulationPyramid =
    let genderFilter gender = transform (filter (FEqual "gender" (Str gender)) [])
        
        yPos extra = position Y ([PName "age", PmType Ordinal, PAxis [], PSort [Descending]] <> extra)
        
        plot gender = [ genderFilter gender
                      , title gender []
                      , mark Bar []
                      , encoding
                          . yPos [PNoTitle | gender == "Male"]
                          . position X
                                     ([PAggregate Sum, PName "people", PmType Quantitative, PAxis [AxTitle "population", AxFormat "s"]]
                                      <> [PSort [Descending] | gender == "Female"])
                          . color ([MName "gender", MmType Nominal, MLegend []]
                                   <> [MScale [SRange (RStrings ["#675193", "#ca8861"])] | gender == "Female"])
                          $ []
                      ]
                      
        labelPlot = [ width 20
                    , viewBackground [VBNoStroke]
                    , mark Text [MAlign AlignCenter]
                    , encoding
                        . yPos []
                        . text [TName "age", TmType Quantitative]
                        $ []
                    ]
        plots = map asSpec [plot "Female", labelPlot, plot "Male"]
    
    in toVegaLite [ description "A population pyramid for the US in 2000."
                  , dataFromUrl "data/population.json" []
                  , transform 
                      . filter (FExpr "datum.year == 2000")
                      . calculateAs "datum.sex == 2 ? 'Female' : 'Male'" "gender"
                      $ []
                  , spacing 0
                  , hConcat plots
                  , configure
                      . configuration (ViewStyle [ViewNoStroke])
                      . configuration (Axis [Grid False])
                      $ []
                  ]

vlShow concatPopulationPyramid

In [59]:
validate concatPopulationPyramidSpec concatPopulationPyramid

Okay

Return to the [Table of Contents](#Table-of-Contents).